<a href="https://colab.research.google.com/github/HamouChicha/House-Price-Prediction-Project/blob/master/Spam_classification_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# Loading the dataset
import pandas as pd

df = pd.read_csv('spam.csv', encoding='Windows-1252')

In [43]:
# Inspecting the dataset...
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [44]:
df.shape

(5572, 5)

In [45]:
df.isnull().sum().sort_values()

,0
v1,0
v2,0
Unnamed: 2,5522
Unnamed: 3,5560
Unnamed: 4,5566


In [46]:
# Keeping only the test and its labels
data = df[['v1', 'v2']]

In [47]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [48]:
# Renaming the columns
data.rename(columns={'v1': 'label', 'v2': 'message'}, inplace=True)

<ipython-input-48-0a842cd6414a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'v1': 'label', 'v2': 'message'}, inplace=True)


In [49]:
data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [50]:
data.isnull().sum().sort_values()

,0
label,0
message,0


In [54]:
# The independent feature
X = list(data['message'])
# The dependent feature (label))
y = list(data['label'])

In [55]:
X[:10]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though",
 "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",
 'Even my brother is not like to speak with me. They treat me like aids patent.',
 "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
 'WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.',
 'Had y

In [56]:
y[:10]

['ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'spam']

In [57]:
# Encoding the labels
pd.get_dummies(y, drop_first = True, dtype='int')

,spam
0,0
1,0
2,1
3,0
4,0
...,...
5567,1
5568,0
5569,0
5570,0


In [58]:
y = list(pd.get_dummies(y, drop_first = True, dtype='int')['spam'])

In [59]:
y[:10]

[0, 0, 1, 0, 0, 1, 0, 0, 1, 1]

In [16]:
# Split the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [17]:
# Importing the tokenizer and instanciate it with the distilbert model
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [19]:
# Tokenize the datasets
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')

In [20]:
train_encodings

{'input_ids': tensor([[ 101, 2053, 2053,  ...,    0,    0,    0],
        [ 101, 1045, 2572,  ...,    0,    0,    0],
        [ 101, 2057, 2939,  ...,    0,    0,    0],
        ...,
        [ 101, 1045, 2001,  ...,    0,    0,    0],
        [ 101, 1024, 1011,  ...,    0,    0,    0],
        [ 101, 2129, 8108,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [30]:
import torch

# Prepare inputs for the trainer
train_inputs = {
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': torch.tensor(y_train)  # or [item['label'] for item in data]
}

test_inputs = {
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': torch.tensor(y_test)  # or [item['label'] for item in data]
}

In [31]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create Dataset
train_dataset = CustomDataset(train_inputs, train_inputs['labels'])
test_dataset = CustomDataset(test_inputs, test_inputs['labels'])

In [23]:
# defining the model hyperparameters for training
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = './results', # output directory
    num_train_epochs = 1, # total number of training epochs
    per_device_train_batch_size = 64, # batch size per device during training  # 8
    per_device_eval_batch_size = 64, # batch size for evaluation   # 16
    warmup_steps = 500, # number of warmup steps for learning rate scheduler
    weight_decay = 0.01, # strength of weight decay
    logging_dir = './logs', # directory for storing logs
    logging_steps = 10
)

In [24]:
# Define and train the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

trainer = Trainer(
    model = model,  # the instanciated Transformers model to be trained
    args = training_args,   # training arguments defined above
    train_dataset = train_dataset,  # training dataset
    eval_dataset = test_dataset   # evaluation dataset
)

trainer.train()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hammouchicha18 (hammouchicha18-university-ibn-zohr) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.686000
20,0.669200
30,0.626700
40,0.553300
50,0.453200
60,0.323300
70,0.231600


TrainOutput(global_step=70, training_loss=0.5061964273452759, metrics={'train_runtime': 5924.7288, 'train_samples_per_second': 0.752, 'train_steps_per_second': 0.012, 'total_flos': 274447094927208.0, 'train_loss': 0.5061964273452759, 'epoch': 1.0})

In [34]:
# Using the model to predict the test dataset
output = trainer.predict(test_dataset)[1]

In [35]:
# The confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, output)
cm

array([[949,   0],
       [  0, 166]])

In [36]:
# The accuracy score
from sklearn.metrics import accuracy_score

accuracy_score(y_test, output)

1.0

In [38]:
# The classification report
from sklearn.metrics import classification_report

print(classification_report(y_test, output))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       949
           1       1.00      1.00      1.00       166

    accuracy                           1.00      1115
   macro avg       1.00      1.00      1.00      1115
weighted avg       1.00      1.00      1.00      1115



In [62]:
# Save the model and the tokenizer
trainer.save_model('spam_model')
tokenizer.save_pretrained('spam_model')

('spam_model/tokenizer_config.json',
 'spam_model/special_tokens_map.json',
 'spam_model/vocab.txt',
 'spam_model/added_tokens.json',
 'spam_model/tokenizer.json')